In [ ]:
""" ▶ tf.keras.applications.vgg16 """
'''
tf.keras.applications 는 이미 만들어진 어플리케이션들이 있는 곳이다.

그 중, vgg16을 가져와서 실습해 볼 예정이다.
'''

In [ ]:
""" ▶ 모델 객체, 레이어 속성 """
'''
 1. 이름으로 레이어 객체 리턴: model.get_layer(name, index).name
 2. 인덱스로 레이어 객체 리턴: model.get_layer(name, index) ;  model.layers[index].name ;  model.layers
 3. 레이어 객체의 속성 및 메소드: tf.keras.layers.Layer의 하위 클래스들을 확인
 4. 조건을 만족하는 레이어 오브젝트 리턴, 설정 값 변경
 5. 모델이나 레이어의 가중치(커널의 가중치)나 바이어스 등의 파라미터 값을 리턴
     5-1. 레이어 파라미터(가중치, 바이어스): get_weights(), weights 속성, trainable_weights, non_trainable_weights, kernel, bias 속성
     5-2. 모델: get_weights(), weights 속성
     5-3. CNN 필터(커널)의 가중치 및 시각화
     
 * 전이학습을 통해서 사용자 모델을 만들고 콜백 핸들러 함수 및 객체를 통해서 튜닝을 할 수 있다.
 CNN, RNN, LSTM_자연어, AE -> (GAN, UNET)
 
 
 tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)
'''

In [4]:
import tensorflow as tf
import pprint 
from tensorflow.keras.applications import VGG16

model = VGG16()#( weights = None )
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#1-2  모델의  name을 지정해서 레이어 대상의 오브젝트를  get_layer()로 리턴  
res = model.get_layer('block4_conv1')
print(res)
print(type(res))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pprint

from tensorflow.keras.applications import ResNet50

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical


from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
#1 데이터 로드  
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#2 데이터 축소   : [ : 1000] 학습용 데이터 1000,  테스트 500개  
train_images = train_images[:1000] 
test_images = test_images[:500]
train_labels = to_categorical(train_labels[:1000], 10)
test_labels = to_categorical(test_labels[:500], 10)

#3  ResNet50  모델 사용, 전체  레이어 동결
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

for layer in base_model.layers:
    layer.trainable = False
    
#4. 분류기 추가. 
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

#model.summary()
#5. 모델 컴파일 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#6. 학습  batch_size  = 32, epochs= 3 
model.fit(train_images, train_labels, batch_size=32, epochs=3, validation_split=0.1)

#7. 평가  
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')
